In [0]:
from pyspark.sql.functions import col, regexp_replace
from pyspark.sql.types import IntegerType
from io import StringIO
import pandas as pd
import boto3
import time
from botocore.exceptions import ClientError

OFFSET=0
OFFSET_FILE="Offset/offset.csv"
BUCKET_NAME = "databricksbucket101"
FILE = "datafiles/transactions.csv"

while True:
    try:
        s3_client = boto3.client('s3')
        s3_client.head_object(Bucket=BUCKET_NAME, Key=OFFSET_FILE)

        print(f"File found at s3://{BUCKET_NAME}/{OFFSET_FILE}. Reading file...")

        s3_path = f"s3://{BUCKET_NAME}/{OFFSET_FILE}"
        response = s3_client.get_object(Bucket=BUCKET_NAME, Key=OFFSET_FILE)

        csv_object = response['Body']

        df_offset = pd.read_csv(csv_object)
        OFFSET=df_offset.iloc[0]['offset']

    except ClientError as e:
      
        if e.response['Error']['Code'] == '404':
            print(f"File not found at s3://{BUCKET_NAME}/{OFFSET_FILE}.")
            OFFSET=0
               
        else:
            print(f"An error occurred: {e}")

    try:

        # Get the object from S3
        response = s3_client.get_object(Bucket=BUCKET_NAME, Key=FILE)

        # The actual data is in the 'Body' of the response, which is a streaming object
        csv_object = response['Body']

        # Read the streaming object directly into pandas
        df_pd = pd.read_csv(csv_object)
        print(df_pd.shape[0],OFFSET)
        print("Successfully read CSV from S3 using boto3.")
        
        df=pd.DataFrame(df_pd.iloc[OFFSET:OFFSET+10000])
        for col in df.select_dtypes(include='object'):
            df[col] = df[col].str.replace("'", "")
        new_offset=OFFSET+10000
        offset_data={"offset":[new_offset]}
        df_offset = pd.DataFrame(offset_data)
        csv_buffer1 = StringIO()
        df.to_csv(csv_buffer1, index=False)
        s3_client.put_object(Bucket=BUCKET_NAME, Key=f"landing_zone/transactions_{new_offset}.csv", Body=csv_buffer1.getvalue())
        csv_buffer2 = StringIO()
        df_offset.to_csv(csv_buffer2, index=False)
        s3_client.put_object(Bucket=BUCKET_NAME,Key="Offset/offset.csv",Body=csv_buffer2.getvalue())

    except Exception as e:
        print(f"An error occurred: {e}")
    if df_pd.shape[0]<=int(OFFSET):
        print("read the whole transactions file")
        break
    time.sleep(5)
    
